In [1]:
!export EB_AGENT_LOGGING_LEVEL='info'
!export EB_AGENT_ACCESS_TOKEN='<your_access_token>'

'<your_access_token>'

# 智能体(Agent)介绍
ERNIE Bot Agent 基于文心大模型的 Function Calling 能力实现了多工具编排和自动调度功能，并且允许工具、插件、知识库等不同组件的混合编排。

除了自动调度，我们未来还将支持更多的编排模式，例如手动编排、半自动编排，为开发者提供更大的灵活性。

在此教程中，主要介绍一个Function Agent的运行过程，Function Agent的编排能力由集成的LLM大语言模型（此处为文心）的函数调用能力驱动，通常由
Agent请求LLM生成一个可以解析的Action，然后Agent根据Action，调用相应的工具或插件，然后完成此操作。

一个智能体的运行大致分为以下三步：
1. 创建需要调用的工具
2. 创建智能体
3. 调用智能体

[Optional]建议将环境变量EB_AGENT_LOGGING_LEVEL设置为info打开相关日志，以方便观察智能体具体的运行过程。


接下来将按照这三步，结合一个具体的实例进行介绍：

实例的背景为创建一个英语学习智能体，帮助用户学习英语单词，包括单词的释义、近义词、反义词和例句并将相关信息添加到单词本。



In [1]:
import os

os.environ["EB_AGENT_LOGGING_LEVEL"] = "INFO"

from pydantic import Field
from typing import Dict, Type, List, Any

from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import HumanMessage, Message, AIMessage, FunctionMessage, SlidingWindowMemory
from erniebot_agent.tools.base import Tool
from erniebot_agent.tools.schema import ToolParameterView
from erniebot_agent.agents import FunctionAgent

## 创建工具
此章节中，主要创建用于英语学习智能体相关的本地工具以便于用户直接体验，不会对tool的相关内容有太多详细的介绍，关于tool的详细内容，请见[local_tool.ipynb](./local_tool.ipynb)。

其中，对于工具的examples的描写，用到了Message模块，详见[memory.ipynb](./memory.ipynb)。

## 定义ChatWithEB工具
在智能体(Agent)中，我们创建一个名为ChatWithEB的工具，该工具用于为用户的单词提供详细的释义，其中可以通过`prompt`或者`system`的方式来控制，达成用户的相关目标（本例中为按要求获取单词相关信息）。

同时`ChatWithEB`工具也是一个推荐的本地工具，加载在智能体当中用于每一个Run中途的交互，从而更好地完成智能体的功能。

例如在本例中，我们通过`ChatWithEB`工具来获取用户输入的单词的详细信息，进而能直接添加进入单词本当中。

如果不使用这个工具，那么用户无法通过一句简单的"帮我添加单词"同时执行这两个操作：需要先通过prompt获得单词相关内容，然后再通过"帮我添加单词"添加相关单词。


In [2]:
class ChatWithEBInputView(ToolParameterView):
    word: str = Field(description="需要获取详细信息的单词")


class ChatWithEBOutputView(ToolParameterView):
    response: str = Field(description="EB回复的对于单词的详细解释")


class ChatWithEB(Tool):
    description: str = "ChatWithEB是一款根据用户的提供的单词，获取一个具体的单词描述、翻译以及例句的工具"
    input_type: Type[ToolParameterView] = ChatWithEBInputView
    ouptut_type: Type[ToolParameterView] = ChatWithEBOutputView

    def __init__(self, llm: ERNIEBot):
        self.llm = llm

    async def __call__(self, word: str) -> Dict[str, str]:
        prompt = f"请告诉我{word}的详细信息，以下面的格式输出（其中单词翻译需要是中文，其他为英语）" "<单词翻译>:\n<近义词>:\n<反义词>:\n<例句>:\n"
        response = await self.llm.chat([HumanMessage(prompt)])
        return {"response": response.content}

    @property
    def examples(self) -> List[Message]:
        return [
            HumanMessage("请帮我把这个单词meticulous存储到单词本中"),
            AIMessage(
                "",
                function_call={
                    "name": self.tool_name,
                    "thoughts": f"用户想把meticulous加入到单词本，我需要先使用{self.tool_name}来获取"
                    "单词'meticulous'的详细信息，然后调用AddWordTool来存储到单词本中",
                    "arguments": '{"word": "meticulous"}',
                },
            ),
        ]


SYSTEM_MESSAGE = "你是一个英语老师，当用户给你一个单词的时候，请你专业而通俗的返回英语的<单词翻译><近义词><反义词><例句>"
llm = ERNIEBot(model="ernie-4.0", system=SYSTEM_MESSAGE)
word_helper_tool = ChatWithEB(llm)

## 定义AddWordTool工具
在此智能体(Agent)中，我们创建一个名为AddWordTool的工具，用于将用户的单词添加至单词本。此工具为`Tool`章节中的示例工具，此处不做过多介绍。详见[local_tool.ipynb](./local_tool.ipynb)。

In [3]:
mimic_response = (
    "<单词翻译>: meticulous中文翻译为“一丝不苟的”、“细致的”、“小心翼翼的”。\n\n"
    "<近义词>: thorough, detailed, precise, conscientious\n\n"
    "<反义词>: careless, sloppy, negligent, cursory\n\n"
    "<例句>: He is known for his meticulous attention to detail in his work."
)


class AddWordInput(ToolParameterView):
    word: str = Field(description="待添加的单词")
    des: str = Field(description="待添加的单词的详细释义")


class AddWordOutput(ToolParameterView):
    result: str = Field(description="表示是否成功将单词成功添加到词库当中")


class AddWordTool(Tool):
    description: str = "添加单词以及它的详细解释到词库当中"
    input_type: Type[ToolParameterView] = AddWordInput
    ouptut_type: Type[ToolParameterView] = AddWordOutput

    def __init__(self) -> None:
        self.word_books = {}
        super().__init__()

    async def __call__(self, word: str, des: str) -> Dict[str, Any]:
        if word in self.word_books:
            return {"result": f"<{word}>单词已经存在，无需添加"}
        self.word_books[word] = des
        words = "\n".join(list(self.word_books.keys()))
        return {"result": f"<{word}>单词已添加成功, 当前单词本中有如下单词：{words}，释义如下{des}"}

    @property
    def examples(self) -> List[Message]:
        return [
            HumanMessage("请帮我把这个单词meticulous存储到单词本中"),
            AIMessage(
                "",
                function_call={
                    "name": self.tool_name,
                    "thoughts": f"用户想把meticulous加入到单词本，我需要先使用ChatWithEB来获取单词'meticulous'的详细信息，"
                    "然后调用AddWordTool来存储到单词本中",
                    "arguments": '{"word": "meticulous"}',
                },
            ),
            FunctionMessage(content=mimic_response, name="ChatWithEB"),
            AIMessage(
                "",
                function_call={
                    "name": self.tool_name,
                    "thoughts": f"用户想把meticulous加入到单词本，我已经使用ChatWithEB来获取单词'meticulous'的详细信息，然后需要调用{self.tool_name}来存储到单词本中",
                    "arguments": f'{{"word": "meticulous", "des":{mimic_response}}}',
                },
            ),
        ]


add_word_tool = AddWordTool()

## 创建智能体
在此步骤中将展示如何创建智能体。

1. 指定调用的文心大模型，通过`enable_multi_step_tool_call`接口打开multi_tool_call功能，开启之后即可通过一个指令进行多次不同工具的调用，通过`max_steps`控制此Run中执行的最大的Step以免一直循环。

2. 为智能体添加默认的`ChatWithEB`和`AddWordTool`工具。

3. [Optional]为智能体指定特殊的`Memory`，默认为`WholeMemory`，详见[memory.ipynb](./memory.ipynb)。

4. [Optional]为智能体指定`SystemMessage`，设置全局的指令。

In [4]:
memory = SlidingWindowMemory(max_round=5)
llm_final = ERNIEBot(model="ernie-3.5", api_type="aistudio", enable_multi_step_tool_call=True)
agent_all = FunctionAgent(llm=llm_final, tools=[word_helper_tool, add_word_tool], memory=memory)

## 调用智能体
在完成了智能体创建之后，通过异步run函数执行指令，并获取返回结果（如果打开了multi_tool_call，那么仅需一句指令即可同时执行单词解释以及添加至单词本两个动作）。

In [5]:
response = await agent_all.run("请帮我把这个单词meticulous存储到单词本中")

INFO - [Run][Start] FunctionAgent is about to start running with input:
请帮我把这个单词meticulous存储到单词本中
INFO - [LLM][Start] ERNIEBot is about to start running with input:
 role: user 
 content: 请帮我把这个单词meticulous存储到单词本中 
INFO - [LLM][End] ERNIEBot finished running with output:
 role: assistant 
 function_call: 
{
  "name": "ChatWithEB",
  "thoughts": "用户想把meticulous加入到单词本，我需要先使用ChatWithEB来获取单词'meticulous'的详细信息，然后调用AddWordTool来存储到单词本中",
  "arguments": "{\"word\":\"meticulous\"}"
} 
INFO - [Tool][Start] ChatWithEB is about to start running with input:
{
  "word": "meticulous"
}
INFO - [Tool][End] ChatWithEB finished running with output:
{
  "response": "<单词翻译>: meticulous 是一个英语单词，中文翻译为“一丝不苟的”、“细致的”、“小心翼翼的”。\n\n<近义词>: 一些与 meticulous 意思相近的词语包括 \"thorough\", \"careful\", \"precise\", \"detailed\", \"diligent\", 和 \"methodical\"。\n\n<反义词>: 与 meticulous 意思相反的词语包括 \"careless\", \"haphazard\", \"negligent\", \"sloppy\", 和 \"cursory\"。\n\n<例句>: \n1. She is known for her meticulous attention to detail 

In [7]:
print(response.text)

根据您的请求，我已经将单词“meticulous”添加到单词本中，并提供了详细的解释和例句。以下是“meticulous”的详细信息：

* 单词翻译：meticulous 是一个英语单词，中文翻译为“一丝不苟的”、“细致的”、“小心翼翼的”。
* 近义词：一些与 meticulous 意思相近的词语包括 “thorough”、“careful”、“precise”、“detailed”、“diligent”和“methodical”。
* 反义词：与 meticulous 意思相反的词语包括 “careless”、“haphazard”、“negligent”、“sloppy”和“cursory”。
* 例句：
1. She is known for her meticulous attention to detail in her work.
2. The meticulous planner always made sure to double-check all the arrangements before the event.
3. He is a meticulous researcher, leaving no stone unturned in his pursuit of the truth.
4. The cleanliness of the house showed the meticulous nature of its owner.
5. She approached the project with a meticulous eye for detail, ensuring that everything was perfect before submitting it to her boss.

如果您还有其他问题或需要进一步了解“meticulous”的用法，请随时告诉我。


In [8]:
response = await agent_all.run("请帮我把gorgeous存储到单词本中")

INFO - [Run][Start] FunctionAgent is about to start running with input:
请帮我把gorgeous存储到单词本中
INFO - [LLM][Start] ERNIEBot is about to start running with input:
 role: user 
 content: 请帮我把gorgeous存储到单词本中 
INFO - [LLM][End] ERNIEBot finished running with output:
 role: assistant 
 function_call: 
{
  "name": "ChatWithEB",
  "thoughts": "用户想把gorgeous加入到单词本，我需要先使用ChatWithEB来获取单词'gorgeous'的详细信息，然后调用AddWordTool来存储到单词本中",
  "arguments": "{\"word\":\"gorgeous\"}"
} 
INFO - [Tool][Start] ChatWithEB is about to start running with input:
{
  "word": "gorgeous"
}
INFO - [Tool][End] ChatWithEB finished running with output:
{
  "response": "<单词翻译>: Gorgeous在中文中可翻译为“华丽的”、“绚丽的”或“壮丽的”。\n\n<近义词>:\n\n1. Magnificent\n2. Splendid\n3. Stunning\n4. Lavish\n5. Exquisite\n\n<反义词>:\n\n1. Plain\n2. Simple\n3. Ordinary\n4. Unsightly\n5. Ugly\n\n<例句>:\n1. She wore a gorgeous red dress that turned heads wherever she went.\n2. The sunset was absolutely gorgeous, with a range of vibrant colors spreading across the sky

In [9]:
print(response.text)

根据您的请求，我已经将单词“gorgeous”添加到单词本中，并提供了详细的解释和例句。以下是“gorgeous”的详细信息：

* 单词翻译：gorgeous 在中文中可翻译为“华丽的”、“绚丽的”或“壮丽的”。
* 近义词：magnificent、splendid、stunning、lavish、exquisite。
* 反义词：plain、simple、ordinary、unsightly、ugly。
* 例句：
1. She wore a gorgeous red dress that turned heads wherever she went.
2. The sunset was absolutely gorgeous, with a range of vibrant colors spreading across the sky.
3. They had a gorgeous wedding ceremony at a luxurious hotel overlooking the ocean.
4. The city's skyline is gorgeous, especially at night when all the buildings are lit up.
5. He bought a gorgeous diamond necklace for his wife's birthday, making her feel like a queen for the day.

如果您还有其他问题或需要进一步了解“gorgeous”的用法，请随时告诉我。


In [10]:
add_word_tool.word_books

{'meticulous': '<单词翻译>: meticulous 是一个英语单词，中文翻译为“一丝不苟的”、“细致的”、“小心翼翼的”。\n\n<近义词>: 一些与 meticulous 意思相近的词语包括 "thorough", "careful", "precise", "detailed", "diligent", 和 "methodical"。\n\n<反义词>: 与 meticulous 意思相反的词语包括 "careless", "haphazard", "negligent", "sloppy", 和 "cursory"。\n\n<例句>: \n1. She is known for her meticulous attention to detail in her work.\n2. The meticulous planner always made sure to double-check all the arrangements before the event.\n3. He is a meticulous researcher, leaving no stone unturned in his pursuit of the truth.\n4. The cleanliness of the house showed the meticulous nature of its owner.\n5. She approached the project with a meticulous eye for detail, ensuring that everything was perfect before submitting it to her boss.',
 'gorgeous': "<单词翻译>: Gorgeous在中文中可翻译为“华丽的”、“绚丽的”或“壮丽的”。\n\n<近义词>:\n\n1. Magnificent\n2. Splendid\n3. Stunning\n4. Lavish\n5. Exquisite\n\n<反义词>:\n\n1. Plain\n2. Simple\n3. Ordinary\n4. Unsightly\n5. Ugly\n\n<例句>:\n1. She wore a gorgeous red dr

## Agent其他相关方法
除了上述主要功能以外，接下来这个章节主要用于介绍一些其他可能会使用到的智能体功能。

### AgentResponse

AgentResponse作为智能体返回的结果，主要包含以下内容：
1. `text`: Agent在本Run中返回的最终文本
2. `steps`: 包括Action

    Agent在步骤中涉及到的文件将存放在Step中，包括输入文件以及输出文件
3. `chat_history`: Agent在本Run中的对话历史
4. `status`: 包括`FINISHED`和`STOPPED`，其中`FINISHED`表示正常结束，`STOPPED`表示智能体因达到最大step数目强行终止

In [11]:
response.chat_history

[<HumanMessage role: 'user', content: '请帮我把gorgeous存储到单词本中', token_count: 1459>,
 <AIMessage role: 'assistant', function_call: {'name': 'ChatWithEB', 'thoughts': "用户想把gorgeous加入到单词本，我需要先使用ChatWithEB来获取单词'gorgeous'的详细信息，然后调用AddWordTool来存储到单词本中", 'arguments': '{"word":"gorgeous"}'}, token_count: 49>,
 <FunctionMessage role: 'function', name: 'ChatWithEB', content: '{"response": "<单词翻译>: Gorgeous在中文中可翻译为“华丽的”、“绚丽的”或“壮丽的”。\\n\\n<近义词>:\\n\\n1. Magnificent\\n2. Splendid\\n3. Stunning\\n4. Lavish\\n5. Exquisite\\n\\n<反义词>:\\n\\n1. Plain\\n2. Simple\\n3. Ordinary\\n4. Unsightly\\n5. Ugly\\n\\n<例句>:\\n1. She wore a gorgeous red dress that turned heads wherever she went.\\n2. The sunset was absolutely gorgeous, with a range of vibrant colors spreading across the sky.\\n3. They had a gorgeous wedding ceremony at a luxurious hotel overlooking the ocean.\\n4. The city\'s skyline is gorgeous, especially at night when all the buildings are lit up.\\n5. He bought a gorgeous diamond necklace for his wi

In [12]:
response.status

'FINISHED'

In [13]:
# Step为该Run中Agent所执行的相关操作，含有tool_name（调用工具的名称），tool_args（工具的入参），input_files（输入的文件）， output_files（输出的文件）
response.steps

[ToolStep(info={'tool_name': 'ChatWithEB', 'tool_args': '{"word":"gorgeous"}'}, result='{"response": "<单词翻译>: Gorgeous在中文中可翻译为“华丽的”、“绚丽的”或“壮丽的”。\\n\\n<近义词>:\\n\\n1. Magnificent\\n2. Splendid\\n3. Stunning\\n4. Lavish\\n5. Exquisite\\n\\n<反义词>:\\n\\n1. Plain\\n2. Simple\\n3. Ordinary\\n4. Unsightly\\n5. Ugly\\n\\n<例句>:\\n1. She wore a gorgeous red dress that turned heads wherever she went.\\n2. The sunset was absolutely gorgeous, with a range of vibrant colors spreading across the sky.\\n3. They had a gorgeous wedding ceremony at a luxurious hotel overlooking the ocean.\\n4. The city\'s skyline is gorgeous, especially at night when all the buildings are lit up.\\n5. He bought a gorgeous diamond necklace for his wife\'s birthday, making her feel like a queen for the day."}', input_files=[], output_files=[]),
 ToolStep(info={'tool_name': 'AddWordTool', 'tool_args': '{"word":"gorgeous","des":"<单词翻译>: Gorgeous在中文中可翻译为“华丽的”、“绚丽的”或“壮丽的”。\\n\\n<近义词>:\\n\\n1. Magnificent\\n2. Splendid\\n3. Stun

### Tool以及Memory相关用法

In [14]:
# 获取当前所有工具
agent_all.get_tools()

[<name: ChatWithEB, description: ChatWithEB是一款根据用户的提供的单词，获取一个具体的单词描述、翻译以及例句的工具>,
 <name: AddWordTool, description: 添加单词以及它的详细解释到词库当中>]

In [15]:
# 查找某个工具
eb_tool = agent_all.get_tool("ChatWithEB")
# 取消某个工具
agent_all.unload_tool(eb_tool)
# 加载某个工具
agent_all.load_tool(eb_tool)

In [16]:
# 获取Agent的对话历史
agent_all.memory.get_messages()

[<HumanMessage role: 'user', content: '请帮我把这个单词meticulous存储到单词本中', token_count: 1194>,
 <AIMessage role: 'assistant', content: '根据您的请求，我已经将单词“meticulous”添加到单词本中，并提供了详细的解释和例句。以下是“meticulous”的详细信息：\n\n* 单词翻译：meticulous 是一个英语单词，中文翻译为“一丝不苟的”、“细致的”、“小心翼翼的”。\n* 近义词：一些与 meticulous 意思相近的词语包括 “thorough”、“careful”、“precise”、“detailed”、“diligent”和“methodical”。\n* 反义词：与 meticulous 意思相反的词语包括 “careless”、“haphazard”、“negligent”、“sloppy”和“cursory”。\n* 例句：\n1. She is known for her meticulous attention to detail in her work.\n2. The meticulous planner always made sure to double-check all the arrangements before the event.\n3. He is a meticulous researcher, leaving no stone unturned in his pursuit of the truth.\n4. The cleanliness of the house showed the meticulous nature of its owner.\n5. She approached the project with a meticulous eye for detail, ensuring that everything was perfect before submitting it to her boss.\n\n如果您还有其他问题或需要进一步了解“meticulous”的用法，请随时告诉我。', token_count: 271>,
 <HumanMessage role: 

In [17]:
# 重置Agent的对话历史
agent_all.reset_memory()
agent_all.memory.get_messages()

[]

# 